## 3. Fonctions utilitaires

In [99]:
# import

import pandas as pd
import os


In [100]:
# Chemin vers les données OULAD
csv_path = '../data/raw/open+university+learning+analytics+dataset/'

# Charger chaque CSV avec gestion de l'encodage
data_dicts = {}
for csv_name in os.listdir(csv_path):
    if csv_name.endswith('.csv'):
        try:
            # Essayer d'abord UTF-8
            data_dicts[csv_name[:-4]] = pd.read_csv(csv_path + csv_name)
        except UnicodeDecodeError:
            # Si échec, utiliser latin-1 (ISO-8859-1)
            data_dicts[csv_name[:-4]] = pd.read_csv(csv_path + csv_name, encoding='latin-1')
            print(f" {csv_name} chargé avec encoding latin-1")
        
print(f"{len(data_dicts)} fichiers CSV chargés")
print(f"Fichiers: {list(data_dicts.keys())}")

7 fichiers CSV chargés
Fichiers: ['assessments', 'courses', 'studentAssessment', 'studentInfo', 'studentRegistration', 'studentVle', 'vle']


## 2. Chargement des données OULAD

In [101]:
import pandas as pd
import numpy as np
import os

## 1. Imports

# Data Preparation - OULAD Dataset

Ce notebook charge les données OULAD, effectue le nettoyage et crée le dataset final pour l'entraînement des modèles ML.

# Prepping the Data for Analysis

In [102]:
# function for merging 2 pandas dataframes

def inner_merge(left_df,right_df,right_cols,on_cols):
    right_df = right_df[right_cols]
    left_df = left_df.merge(right_df, on = on_cols)
    return left_df.drop_duplicates()

We want to predict a student's final result based on their early performance. By performance, we mean 2 things: academic performance, measured via the students' scores, and academic engagement, measured via the number of times the students click on course content. The three variables below control how many days into a semester we want to include for our analysis.  

In [103]:
score_deadline = 90
click_deadline = 90
withdraw_deadline = 90

Below we have a function that creates a dataset that contains each student's average assignment score within the first `score_deadline` days. Another function to create a dataset that contains each student's average number of clicks for each resource category within the first `click_deadline` days. Then a function to merge the two datasets together, as well as remove students who withdrew before `withdraw_deadline`. Since our analysis only concerns with Pass/Fail, we merge Distinction final results into Pass, and merge Withdrawn into Fail. The idea is that after a certain amount of time, the only people who withdraw are the people who are concerned with their performance, and are less likely to withdraw due to non-academic related reasons. 

In [104]:
# score data

def create_score_df(score_deadline):

    df1 = data_dicts['assessments'].copy()
    df2 = data_dicts['studentAssessment'].copy()
    
    # Convertir les colonnes en numérique
    df1['date'] = pd.to_numeric(df1['date'], errors='coerce')
    df1['weight'] = pd.to_numeric(df1['weight'], errors='coerce')
    df2['date_submitted'] = pd.to_numeric(df2['date_submitted'], errors='coerce')
    df2['score'] = pd.to_numeric(df2['score'], errors='coerce')
    df2['is_banked'] = pd.to_numeric(df2['is_banked'], errors='coerce')

    score_df = inner_merge(df1,
                           df2,
                           df2.columns,
                           ['id_assessment'],
                          )

    score_df = score_df[score_df['date'] < score_deadline]

    score_df = score_df[score_df['assessment_type'] != 'Exam']

    score_df = score_df.dropna(subset = ['score']) 

    score_df.isnull().sum()

    score_df['assessment_type'].value_counts()

    # Calculer la moyenne seulement sur les colonnes numériques
    score_df = score_df.groupby(['code_module', 'code_presentation', 'id_student'], as_index=False).agg({
        'score': 'mean',
        'date': 'mean',
        'weight': 'mean',
        'date_submitted': 'mean',
        'is_banked': 'mean',
        'id_assessment': 'mean'
    })
    
    score_df = score_df.rename(columns = {'score': f'mean_score_day{score_deadline}'})
    score_df = score_df.drop(['date', 'weight', 
                              'date_submitted', 'is_banked',
                              'id_assessment'],
                             axis = 1)
    return score_df

In [105]:
# click data

def create_click_df(click_deadline):

    clicks = data_dicts['studentVle'].copy()
    
    # Convertir les colonnes en numérique
    clicks['date'] = pd.to_numeric(clicks['date'], errors='coerce')
    clicks['sum_click'] = pd.to_numeric(clicks['sum_click'], errors='coerce')

    clicks = inner_merge(clicks,
                         data_dicts['vle'],
                         ['id_site','code_module','code_presentation','activity_type'],
                         ['id_site','code_module','code_presentation'],
                        )

    clicks = clicks.drop('id_site', axis = 1)

    def clicks_xx(clicks,xx):
        temp = clicks[clicks['date'] <= xx]
        temp = temp.drop('date', axis = 1)
        temp = temp.groupby(['code_module','code_presentation','id_student','activity_type']).mean()
        temp = temp.rename(columns = {'sum_click': f'sum_click{xx} mean'})
        temp = temp.reset_index() # this fills out the missing columns for merging later 
        return temp

    click_data = pd.pivot_table(data = clicks_xx(clicks,click_deadline), 
                                index = ['code_module','code_presentation','id_student'],
                                columns = 'activity_type', 
                                values = [f'sum_click{click_deadline} mean'],
                                fill_value = 0,
                               ).reset_index()

    # get rid of multi index
    click_data = pd.concat([click_data['code_module'],
                            click_data['code_presentation'],
                            click_data['id_student'], 
                            click_data[f'sum_click{click_deadline} mean']], axis=1)
    return click_data

In [106]:
# merge score_df and click_df
def create_final_df(withdraw_deadline, score_df, click_df):

    final_df = inner_merge(click_df, # merge with StudentInfo
                           data_dicts['studentInfo'],
                           data_dicts['studentInfo'].columns,
                           ['code_module', 'code_presentation', 'id_student'],
                          )

    final_df = final_df.replace('Distinction','Pass') # merge Pass and Distinction

    # Copier studentRegistration et convertir date_unregistration
    student_reg = data_dicts['studentRegistration'].copy()
    student_reg['date_unregistration'] = pd.to_numeric(student_reg['date_unregistration'], errors='coerce')
    student_reg['date_registration'] = pd.to_numeric(student_reg['date_registration'], errors='coerce')
    
    final_df = inner_merge(final_df, 
                           student_reg,
                           ['code_module', 'code_presentation', 'id_student', 'date_unregistration'],
                           ['code_module', 'code_presentation', 'id_student'],
                          )

    final_df = final_df[(final_df['final_result'] != 'Withdrawn') | # remove people who withdrew before withdraw_deadline
                       (final_df['date_unregistration'] > withdraw_deadline)]

    final_df = final_df.reset_index()

    final_df = final_df.drop(['date_unregistration','index'],axis = 1)

    final_df = inner_merge(final_df,
                           score_df,
                           score_df.columns,
                           ['code_module', 'code_presentation', 'id_student'])
    
    final_df = final_df.replace('Withdrawn','Fail') # merge Withdrawn to Fail

    return final_df

In [107]:
score_df = create_score_df(score_deadline)
click_df = create_click_df(click_deadline)
final_df = create_final_df(withdraw_deadline, score_df, click_df)
final_df.head()

,code_module,code_presentation,id_student,dataplus,dualpane,externalquiz,forumng,glossary,homepage,htmlactivity,...,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,mean_score_day90
0,AAA,2013J,11391,0.0,0.0,0.0,3.514286,0.0,3.478261,0.0,...,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,81.5
1,AAA,2013J,28400,0.0,0.0,0.0,2.988889,0.0,5.194444,0.0,...,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,69.0
2,AAA,2013J,31604,0.0,0.0,0.0,2.955882,1.0,4.340000,0.0,...,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,71.5
3,AAA,2013J,32885,0.0,0.0,0.0,3.127660,2.0,3.600000,0.0,...,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,49.5
4,AAA,2013J,38053,7.0,0.0,0.0,3.457364,0.0,4.438596,0.0,...,M,Wales,A Level or Equivalent,80-90%,35-55,0,60,N,Pass,74.0


In [108]:
# Créer le dossier processed s'il n'existe pas
import os
os.makedirs('../data/processed', exist_ok=True)

# Sauvegarder le DataFrame final
output_path = '../data/processed/final_df.csv'
final_df.to_csv(output_path, index=False, encoding='utf-8')
print(f"Données sauvegardées : {output_path}")
print(f"Dimensions : {final_df.shape}")
print(f"Colonnes : {list(final_df.columns)}")

Données sauvegardées : ../data/processed/final_df.csv
Dimensions : (23743, 32)
Colonnes : ['code_module', 'code_presentation', 'id_student', 'dataplus', 'dualpane', 'externalquiz', 'forumng', 'glossary', 'homepage', 'htmlactivity', 'oucollaborate', 'oucontent', 'ouelluminate', 'ouwiki', 'page', 'questionnaire', 'quiz', 'repeatactivity', 'resource', 'sharedsubpage', 'subpage', 'url', 'gender', 'region', 'highest_education', 'imd_band', 'age_band', 'num_of_prev_attempts', 'studied_credits', 'disability', 'final_result', 'mean_score_day90']


## 6. Sauvegarde des données préparées

Sauvegarde du DataFrame final pour utilisation dans les notebooks suivants.